In [58]:
# 1200 * 910 the standers, pls edit below for test image dim
configuration = {

    "resize": (224, 224),

    "crop_positions": {
        
        "Q1": {"x1": 772, "y1": 91, "x2": (772+360), "y2": (91+360), "center": (967,275), "radius":143, "apply_circular_mask":True},
        
        "Q2": {"x1": 410, "y1": 90, "x2": (410+360), "y2": (90+360), "center": (592, 275), "radius":143, "apply_circular_mask":True},
        
        "Q3": {"x1": 410, "y1": 467, "x2": (410+360), "y2": (467+360), "center": (592,652), "radius":143, "apply_circular_mask":True},
        
        "Q4": {"x1": 771, "y1": 467, "x2": (771+360), "y2": (467+360), "center": (967,652), "radius":143, "apply_circular_mask":True},
        
        "text_panel": {"x1": 10, "y1": 200, "x2": (10+315), "y2": (200+625)}
    }
}

In [59]:
from xgboost import XGBClassifier
import joblib


class XGBoostClassifier:

    def __init__(self,
                 n_estimators=400,
                 max_depth=6,
                 learning_rate=0.05,
                 subsample=0.9,
                 colsample_bytree=0.9,
                 random_state=42):

        self.model = XGBClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            objective='binary:logistic',
            eval_metric='logloss',
            random_state=random_state
        )

    # ---------------------------------------
    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    # ---------------------------------------
    def predict(self, X):
        return self.model.predict(X)

    # ---------------------------------------
    def predict_proba(self, X):
        return self.model.predict_proba(X)

    # ---------------------------------------
    def save(self, path):
        joblib.dump(self.model, path)

    # ---------------------------------------
    def load(self, path):
        self.model = joblib.load(path)
        return self


In [66]:
import json
import colorsys
import cv2
import os
import numpy as np
import torch
from colorthief import ColorThief


class VisionModule:
    
    def __init__(self, feature_extraction_model, configuration):
        self.handcraft_features = None
        self.feature_extraction_model = feature_extraction_model
        self.configuration = configuration
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.label_map = {  
            "normal": 0,
            "Keratoconus": 1
        }
        self.test_label_map = {  
            0: "normal",
            1: "Keratoconus"
        }
        
        
    def crop(self, img, img_name="UNKNOWN", save_dir=None):

        crops = {}
        crop_cfg = self.configuration["crop_positions"]

        for key, cfg in crop_cfg.items():
            
            # --- 1. Crop the region ---
            x1, y1, x2, y2 = cfg["x1"], cfg["y1"], cfg["x2"], cfg["y2"]
            crop_img = img[y1:y2, x1:x2].copy()
            
            # --- 2. Apply circular mask ONLY if mask_radius_factor exists ---
            if cfg.get("apply_circular_mask", False) is True:
                radius = cfg.get("radius", None)
                center = cfg.get("center", None)
                center = (center[0] - x1, center[1] - y1)
                crop_img = self._apply_circular_mask(
                    crop_img,
                    radius,
                    center
                )

            # --- 3. Optionally save cropped image ---
            if save_dir is not None and key != "text_panel":
                os.makedirs(save_dir, exist_ok=True)
                save_path = os.path.join(save_dir, f"{img_name}_{key}.png")
                cv2.imwrite(save_path, cv2.cvtColor(crop_img, cv2.COLOR_RGB2BGR))

            # --- 4. Add to output dictionary ---
            crops[key] = crop_img

        return crops


    def _apply_circular_mask(self, img, radius, center):
        h, w = img.shape[:2]
        
        if center is not None:
            center = (int(center[0]), int(center[1]))
        else:
            center = (int(w // 2), int(h // 2))
            
        if radius is not None:
            radius = int(radius)
        else:
            radius = int(min(h, w) * 0.9 / 2)
    
        mask = np.zeros((h, w), dtype=np.uint8)
    
        # make white circle on a center
        cv2.circle(mask, center, radius, (255, 255, 255), -1)
    
        mask = mask.astype(float) / 255.0
        
        fill_color = [0.485*255, 0.456*255, 0.406*255]
        fill_color = np.array(fill_color, dtype=np.float32)
        
        fill_img = np.ones_like(img, dtype=np.float32) * fill_color
    
        result = img.astype(float) * mask[..., None] + fill_img * (1 - mask[..., None])
    
        return result.astype(np.uint8)
    
    
    def process_crops(self, crops):
        processed = {}
        
        for key, img in crops.items():
            # Skip text panel for CNN
            if key == "text_panel":
                processed[key] = img
                continue
            
            processed[key] = self._preprocess_for_cnn(img)
        
        return processed
    
    
    def preprocess_for_cnn(self, cropped_img):
        # resize
        size = self.configuration["resize"]
        img = cv2.resize(cropped_img, size)
    
        # convert to float32 0–1
        img = img.astype(np.float32) / 255.0
    
        # ImageNet norm (change if you use custom model)
        mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
        std  = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    
        img = (img - mean) / std
    
        # HWC → CHW
        img = np.transpose(img, (2, 0, 1))
    
        return img
    
      
    def run_image_preprocessing(self, image_path):

        class_dir = os.path.dirname(image_path)  # e.g. dataset/normal/images
        base_dir = os.path.dirname(class_dir)    # e.g. dataset/normal
    
        save_text_dir   = os.path.join(base_dir, "text_panels")
        save_crops_dir  = os.path.join(base_dir, "crops")
    
        os.makedirs(save_text_dir, exist_ok=True)
        os.makedirs(save_crops_dir, exist_ok=True)
    
        img_name = os.path.splitext(os.path.basename(image_path))[0]
    
        bgr = cv2.imread(image_path)
        if bgr is None:
            raise FileNotFoundError(f"Could not read: {image_path}")
    
        img = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    
        crops = self.crop(img, img_name=img_name, save_dir=save_crops_dir)
    
        # Save all crops for debugging / training
        for key, crop_rgb in crops.items():
            if key == "text_panel":
                continue
            else:
                out_path = os.path.join(save_crops_dir, f"{img_name}_{key}.png")
                cv2.imwrite(out_path, cv2.cvtColor(crop_rgb, cv2.COLOR_RGB2BGR))
    
        # Save text panel for OCR
        text_crop_rgb = crops["text_panel"]
        text_path = os.path.join(save_text_dir, f"{img_name}_textpanel.png")
    
        cv2.imwrite(text_path, cv2.cvtColor(text_crop_rgb, cv2.COLOR_RGB2BGR))
    
        return 
    

    def run_vision_preprocessing(self, folder_path):

        exts = (".jpg", ".jpeg", ".png", ".bmp")
        all_images = [
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.lower().endswith(exts)
        ]
    
        print(f"[INFO] Found {len(all_images)} images in: {folder_path}")
    
       # parent class folder (normal or Keratoconus)
        class_dir = os.path.dirname(folder_path)
    
        # resolved main output dirs
        crops_dir = os.path.join(class_dir, "crops")
        # deep_dir = os.path.join(class_dir, "deep_features")
        # hand_dir = os.path.join(class_dir, "handcraft_features")
        text_panel_dir = os.path.join(class_dir, "text_panels")
        
        processed_list = []

        for img_path in all_images:
            try:
                print(f"[INFO] Processing: {img_path}")
                self.run_image_preprocessing(img_path)
                
                processed_list.append(os.path.basename(img_path))
    
            except Exception as e:
                print(f"[ERROR] Failed on {img_path}: {e}")
                continue
    
        print("[INFO] Completed folder processing.")
        
        return {
            "crops": crops_dir,
            "text_panel_dir": text_panel_dir,
            "processed_images": processed_list
        }
    
    
    def extract_deep_features(self, tensor, save_dir=None, img_name=None, quadrant=None):

        t = torch.from_numpy(tensor).unsqueeze(0).float().to(self.device)
        # shape (1,3,224,224) for resnet 50
    
        with torch.no_grad():
            deep_features = self.feature_extraction_model(t).cpu().numpy().flatten()
            # === OPTIONAL SAVE ===
        if save_dir is not None:
    
            # Ensure folder exists
            os.makedirs(save_dir, exist_ok=True)
    
            # Build filename
            # Example: "image001_Q1.npy"
            if img_name is None:
                img_name = "unknown"
    
            if quadrant is None:
                quadrant = "QX"
    
            filename = f"{img_name}_{quadrant}.npy"
            output_path = os.path.join(save_dir, filename)
    
            # Save features
            np.save(output_path, deep_features)
        return deep_features
    

    def handcrafted_features(self, cropped_img_pth, save_dir=None, img_name=None, quadrant=None):
            
        colors_platte = ColorThief(cropped_img_pth)
        dominant_colors = colors_platte.get_palette(color_count=2)
        # print(dominant_colors)
        
        ignored_colors = [
            (0, 0, 0),
            (255, 255, 255),
            (int(0.485*255), int(0.456*255), int(0.406*255))  # fill mask base color
        ]
        tol = 15
        cleaned_colors = []
        
        for R, G, B in dominant_colors:
            bad = False
            for ir, ig, ib in ignored_colors:
                if (abs(R - ir) <= tol and
                    abs(G - ig) <= tol and
                    abs(B - ib) <= tol):
                    bad = True
                    break
        
            if not bad:
                cleaned_colors.append((R, G, B))
        
        if len(cleaned_colors) == 0:
            cleaned_colors = [(0,0,0)]
        
        R, G, B = cleaned_colors[0]
        r, g, b = R/255.0, G/255.0, B/255.0
        
        h, s, v = colorsys.rgb_to_hsv(r, g, b)
        
        dom_h = h
        dom_s = s
        dom_v = v
        
        # ---- FIXED BGR → RGB for cv2 ----
        cropped_img = cv2.imread(cropped_img_pth)
        cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
    
        img = cropped_img.astype(np.float32) / 255.0
        hsv = cv2.cvtColor(cropped_img, cv2.COLOR_RGB2HSV).astype(np.float32)
    
        # Create mask of "valid" pixels matrix/grid
        valid_mask = np.ones((img.shape[0], img.shape[1]), dtype=bool)
    
        for ir, ig, ib in ignored_colors:
            if (ir, ig, ib) == ignored_colors[2]:
                # This is the FILL MASK → use tolerance
                invalid = (
                    (np.abs(cropped_img[:,:,0] - ir) < tol) &
                    (np.abs(cropped_img[:,:,1] - ig) < tol) &
                    (np.abs(cropped_img[:,:,2] - ib) < tol)
                )
            else:
                # Black & white → exact match
                invalid = (
                    (cropped_img[:,:,0] == ir) &
                    (cropped_img[:,:,1] == ig) &
                    (cropped_img[:,:,2] == ib)
                )
        
            valid_mask[invalid] = False
    
        # Valid pixels only
        valid_hsv = hsv[valid_mask]
    
        # Intensity Features (based on V channel)
        V = valid_hsv[:,2] / 255.0
    
        avg_intensity = float(V.mean())
        std_intensity = float(V.std())
    
        # Center vs Periphery intensities, the center sharpness
        h_im, w_im = cropped_img.shape[:2]
        cy, cx = h_im//2, w_im//2
        R = min(cx, cy)
    
        r_center = int(R * 0.3)
        r_mid = int(R * 0.60)
    
        Y, X = np.ogrid[:h_im, :w_im]
        dist = np.sqrt((X - cx)**2 + (Y - cy)**2)
    
        center_mask = dist < r_center
        periphery_mask = (dist > r_mid) & (dist < R)
    
        center_vals = img[center_mask][:, :].mean() if center_mask.any() else 0.0
        periphery_vals = img[periphery_mask][:, :].mean() if periphery_mask.any() else 1e-6
        
        center_intensity = float(center_vals)
        periphery_intensity = float(periphery_vals)
        center_periphery_ratio = float(center_intensity / periphery_intensity)
    
        # Quadrant splits and asymmetry ratios
        top = img[:h_im//2,:,:].mean()
        bottom = img[h_im//2:,:,:].mean()
        left = img[:, :w_im//2, :].mean()
        right = img[:, w_im//2:, :].mean()
    
        inferior_superior_ratio = float(bottom / (top + 1e-6))
        left_right_ratio = float(left / (right + 1e-6))
    
        diag1 = img[:h_im//2, :w_im//2, :].mean() - img[h_im//2:, w_im//2:, :].mean()
        diag2 = img[:h_im//2, w_im//2:, :].mean() - img[h_im//2:, :w_im//2, :].mean()
    
        diag1_difference = float(diag1)
        diag2_difference = float(diag2)
    
        radial_symmetry = float(
            abs(top - bottom) +
            abs(left - right) +
            abs(diag1) +
            abs(diag2)
        )
    
        # ----- PACK RESULTS -----
        features = {
            "dom_h": dom_h,
            "dom_s": dom_s,
            "dom_v": dom_v,
    
            "avg_intensity": avg_intensity,
            "std_intensity": std_intensity,
    
            "center_intensity": center_intensity,
            "periphery_intensity": periphery_intensity,
            "center_periphery_ratio": center_periphery_ratio,
    
            "inferior_superior_ratio": inferior_superior_ratio,
            "left_right_ratio": left_right_ratio,
    
            "diag1_difference": diag1_difference,
            "diag2_difference": diag2_difference,
    
            "radial_symmetry": radial_symmetry
        }
        self.handcraft_features = features
    
        # ============== OPTIONAL SAVE ==============
        if save_dir is not None:
    
            os.makedirs(save_dir, exist_ok=True)
    
            if img_name is None:
                img_name = "unknown"
    
            if quadrant is None:
                quadrant = "QX"
    
            filename = f"{img_name}_{quadrant}.json"
            output_path = os.path.join(save_dir, filename)
    
            # save handcrafted features as JSON
            with open(output_path, "w") as f:
                json.dump(features, f, indent=4)
    
        return features
    

    # def run_vision_training(self, outputs, save_path="saved_models"):
    # 
    # 
    #     X = []
    #     y = []
    # 
    #     label_map = self.label_map
    # 
    #     os.makedirs(save_path, exist_ok=True)
    # 
    #     # ------------------------------------------
    #     # LOOP OVER CLASSES
    #     # ------------------------------------------
    #     for class_name, info in outputs.items():
    # 
    #         class_label = label_map[class_name]
    # 
    #         deep_dir = info["deep_features_dir"]
    #         hand_dir = info["handcraft_features_dir"]
    #         processed_list = info["processed_images"]
    # 
    #         # FUSION METADATA OUTPUT DIR
    #         fusion_out_dir = os.path.join(
    #             os.path.dirname(deep_dir), "metadata_fusion"
    #         )
    #         os.makedirs(fusion_out_dir, exist_ok=True)
    # 
    #         print(f"[INFO] Generating fusion metadata for: {class_name}")
    # 
    #         # ------------------------------------------
    #         # PROCESS EACH IMAGE
    #         # ------------------------------------------
    #         for img_id in processed_list:
    #             
    #             img_id = os.path.splitext(img_id)[0]
    # 
    #             quad_vecs = []
    #             quad_hand_dict = {}
    # 
    #             # -----------------------
    #             # load quadrant features
    #             # -----------------------
    #             for q in ["Q1", "Q2", "Q3", "Q4"]:
    # 
    #                 npy_path = os.path.join(deep_dir, f"{img_id}_{q}.npy")
    #                 j_path  = os.path.join(hand_dir, f"{img_id}_{q}.json")
    # 
    #                 # load deep features
    #                 deep = np.load(npy_path).astype(np.float32)
    # 
    #                 # load handcrafted
    #                 with open(j_path,"r") as f:
    #                     hand = json.load(f)
    # 
    #                 # handcrafted vector
    #                 hand_vec = np.array([
    #                     hand["dom_h"], hand["dom_s"], hand["dom_v"],
    #                     hand["avg_intensity"], hand["std_intensity"],
    #                     hand["center_intensity"], hand["periphery_intensity"],
    #                     hand["center_periphery_ratio"],
    #                     hand["inferior_superior_ratio"],
    #                     hand["left_right_ratio"],
    #                     hand["diag1_difference"],
    #                     hand["diag2_difference"],
    #                     hand["radial_symmetry"]
    #                 ], dtype=np.float32)
    # 
    #                 quad_vecs.append(np.concatenate([deep, hand_vec], axis=0))
    #                 quad_hand_dict[q] = hand
    # 
    #             # -------------------------
    #             # FUSE quadrants
    #             # -------------------------
    #             fused_vec = np.mean(np.stack(quad_vecs), axis=0)
    # 
    #             X.append(fused_vec)
    #             y.append(class_label)
    # 
    #             # -------------------------
    #             # TEMPORARY fake prediction
    #             # (during training we don't know yet)
    #             # -------------------------
    #             fusion_metadata = {
    #                 "class_name": class_name,
    #                 "embedding": fused_vec.tolist(),
    #                 "Qs": quad_hand_dict,
    #             }
    # 
    #             # save file
    #             fusion_path = os.path.join(fusion_out_dir, f"{img_id}.json")
    #             with open(fusion_path, "w") as f:
    #                 json.dump(fusion_metadata, f, indent=4)
    # 
    #         print(f"[INFO] Fusion metadata saved → {fusion_out_dir}")
    # 
    #     # ------------------------------------------
    #     # TRAIN CLASSIFIER
    #     # ------------------------------------------
    #     X = np.stack(X)
    #     y = np.array(y)
    # 
    #     print(f"[INFO] Training classifier on {X.shape[0]} samples, dim={X.shape[1]}")
    # 
    #     self.classifier.fit(X, y)
    # 
    #     # ------------------------------------------
    #     # SAVE MODELS
    #     # ------------------------------------------
    #     model_path = os.path.join(save_path, "xgboost_vision_model.json")
    #     self.classifier.save(model_path)
    # 
    #     torch.save(self.feature_extraction_model.state_dict(), os.path.join(save_path, "feature_extractor.pth"))
    # 
    #     print("[INFO] Models saved successfully.")
    # 
    #     return True
    
    def split(self):
        pass
    
    # def vision_test(self, image_pth, save_path, model_dir="saved_models"):
    #     
    #     if save_path is not None:
    #         # Ensure parent folder exists
    #         os.makedirs(save_path, exist_ok=True)
    #     
    #         # Subfolders
    #         crops_save_dir = os.path.join(save_path, "crops")
    #         hand_save_dir  = os.path.join(save_path, "handcraft_features")
    #     
    #         os.makedirs(crops_save_dir, exist_ok=True)
    #         os.makedirs(hand_save_dir, exist_ok=True)
    #     
    #         # Extract image name without extension
    #         img_name = os.path.splitext(os.path.basename(image_pth))[0]
    #     
    #     else:
    #         crops_save_dir = None
    #         hand_save_dir  = None
    #         img_name = None
    # 
    #         
    #     # -------------------------------------------------
    #     # 1. Load image
    #     # -------------------------------------------------
    #     img = cv2.imread(image_pth)
    #     if img is None:
    #         raise ValueError(f"[ERROR] Cannot load image: {image_pth}")
    #     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 
    #     # -------------------------------------------------
    #     # 2. Crop quadrants
    #     # -------------------------------------------------
    #     crops = self.crop(img, img_name, crops_save_dir)
    # 
    #     # -------------------------------------------------
    #     # 3. Preprocess quadrants for CNN (skip text panel)
    #     # -------------------------------------------------
    #     processed = self.process_crops(crops)
    # 
    #     quad_vecs = []
    #     quad_output = {}
    # 
    #     # -------------------------------------------------
    #     # 4. Deep + Handcrafted features
    #     # -------------------------------------------------
    #     for q in ["Q1", "Q2", "Q3", "Q4"]:
    #         tensor = processed[q]   # (3,224,224)
    #         
    #         if save_path is not None:
    #             quadrant = q
    #         else:
    #             quadrant = None
    #             
    #         # deep features
    #         deep_vec = self.extract_deep_features(
    #             tensor=tensor,
    #             save_dir=None,
    #             img_name=None,
    #             quadrant=None
    #         )
    # 
    #         # handcrafted features
    #         # first save crop to temp file for the ColorThief
    #         tmp = os.path.join(model_dir, f"_tmp_{q}.png")
    #         cv2.imwrite(tmp, cv2.cvtColor(crops[q], cv2.COLOR_RGB2BGR))
    # 
    #         hand = self.handcrafted_features(
    #             cropped_img_pth=tmp,
    #             save_dir=hand_save_dir,
    #             img_name=img_name,
    #             quadrant=quadrant
    #         )
    #         os.remove(tmp)
    # 
    #         # convert handcraft dict → numeric vector
    #         hand_vec = np.array([
    #             hand["dom_h"], hand["dom_s"], hand["dom_v"],
    #             hand["avg_intensity"], hand["std_intensity"],
    #             hand["center_intensity"], hand["periphery_intensity"],
    #             hand["center_periphery_ratio"],
    #             hand["inferior_superior_ratio"],
    #             hand["left_right_ratio"],
    #             hand["diag1_difference"],
    #             hand["diag2_difference"],
    #             hand["radial_symmetry"]
    #         ], dtype=np.float32)
    # 
    #         # combine deep + hand
    #         fused_q = np.concatenate([deep_vec, hand_vec], axis=0)
    #         quad_vecs.append(fused_q)
    # 
    #         quad_output[q] = {
    #             # "deep_features": deep_vec.tolist(),
    #             "handcrafted_features": hand
    #         }
    # 
    #     # -------------------------------------------------
    #     # 5. Fuse quadrants (mean pooling)
    #     # -------------------------------------------------
    #     fused_embedding = np.mean(np.stack(quad_vecs), axis=0).reshape(1, -1)
    # 
    #     # -------------------------------------------------
    #     # 6. Load XGBoost model
    #     # -------------------------------------------------
    #     model_path = os.path.join(model_dir, "xgboost_vision_model.json")
    #     self.classifier.load(model_path)
    # 
    #     # -------------------------------------------------
    #     # 7. Predict
    #     # -------------------------------------------------
    #     probs = self.classifier.predict_proba(fused_embedding)[0]
    #     pred_idx = int(np.argmax(probs))
    # 
    #     # load metadata to invert mapping
    #     pred_label = self.test_label_map[pred_idx]
    # 
    #     # -------------------------------------------------
    #     # 8. Build return dict
    #     # -------------------------------------------------
    #     output = {
    #         "prediction": pred_label,
    #         "confidence": float(probs[pred_idx]),
    #         "probabilities": probs.tolist(),
    #         "quadrant_features": quad_output
    #     }
    # 
    #     return output


### create the backbone feature extractor

In [67]:
from torch import nn
import torchvision.models as models

backbone = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

feature_extractor = nn.Sequential(*list(backbone.children())[:-1])
feature_extractor = feature_extractor.to("cuda" if torch.cuda.is_available() else "cpu")

In [68]:
vision_obj = VisionModule(feature_extraction_model=feature_extractor, configuration=configuration)

In [63]:
dataset_paths = [r"dataset/Keratoconus/images", r"dataset/normal/images"] 

outputs={}

for dataset_path in dataset_paths:
    class_name = os.path.basename(os.path.dirname(dataset_path))
    outputs[class_name] = vision_obj.run_vision_preprocessing(dataset_path)


[INFO] Found 10 images in: dataset/Keratoconus/images
[INFO] Processing: dataset/Keratoconus/images\53.jpg
[INFO] Processing: dataset/Keratoconus/images\55.jpg
[INFO] Processing: dataset/Keratoconus/images\56.jpg
[INFO] Processing: dataset/Keratoconus/images\57.jpg
[INFO] Processing: dataset/Keratoconus/images\59.jpg
[INFO] Processing: dataset/Keratoconus/images\61.jpg
[INFO] Processing: dataset/Keratoconus/images\62.jpg
[INFO] Processing: dataset/Keratoconus/images\65.jpg
[INFO] Processing: dataset/Keratoconus/images\66.jpg
[INFO] Processing: dataset/Keratoconus/images\67.jpg
[INFO] Completed folder processing.
[INFO] Found 10 images in: dataset/normal/images
[INFO] Processing: dataset/normal/images\207.jpg
[INFO] Processing: dataset/normal/images\208.jpg
[INFO] Processing: dataset/normal/images\209.jpg
[INFO] Processing: dataset/normal/images\210.jpg
[INFO] Processing: dataset/normal/images\211.jpg
[INFO] Processing: dataset/normal/images\212.jpg
[INFO] Processing: dataset/normal/ima

In [64]:
print(outputs)

{'Keratoconus': {'crops': 'dataset/Keratoconus\\crops', 'text_panel_dir': 'dataset/Keratoconus\\text_panels', 'processed_images': ['53.jpg', '55.jpg', '56.jpg', '57.jpg', '59.jpg', '61.jpg', '62.jpg', '65.jpg', '66.jpg', '67.jpg']}, 'normal': {'crops': 'dataset/normal\\crops', 'text_panel_dir': 'dataset/normal\\text_panels', 'processed_images': ['207.jpg', '208.jpg', '209.jpg', '210.jpg', '211.jpg', '212.jpg', '213.jpg', '214.jpg', '215.jpg', '216.jpg']}}


In [69]:
for class_name, info in outputs.items():

        print(f"\n[INFO] Processing class: {class_name}")

        crops_dir = info["crops"]                   # where Q1/Q2/Q3/Q4 crops are
        processed_images = info["processed_images"] # list of filenames
        class_base = os.path.dirname(crops_dir)     # ../normal or ../Keratoconus

        # Create output dirs
        deep_dir = os.path.join(class_base, "deep_features")
        hand_dir = os.path.join(class_base, "handcraft_features")

        os.makedirs(deep_dir, exist_ok=True)
        os.makedirs(hand_dir, exist_ok=True)

        for img_name in processed_images:

            img_id = os.path.splitext(img_name)[0]

            print(f"[INFO] Extracting features for: {img_id}")

            for q in ["Q1", "Q2", "Q3", "Q4"]:

                crop_path = os.path.join(crops_dir, f"{img_id}_{q}.png")

                if not os.path.exists(crop_path):
                    print(f"[WARN] Missing crop: {crop_path}")
                    continue

                # -----------------------------
                # Load crop
                # -----------------------------
                crop = cv2.imread(crop_path)
                crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)

                # -----------------------------
                # Deep features (CNN)
                # -----------------------------
                cnn_ready = vision_obj.preprocess_for_cnn(crop)
                deep_vec = vision_obj.extract_deep_features(
                    tensor=cnn_ready,
                    save_dir=deep_dir,
                    img_name=img_id,
                    quadrant=q
                )

                # -----------------------------
                # Handcrafted features
                # -----------------------------
                vision_obj.handcrafted_features(
                    cropped_img_pth=crop_path,
                    save_dir=hand_dir,
                    img_name=img_id,
                    quadrant=q
                )

        print(f"[INFO] Completed extracting features for class {class_name}")


[INFO] Processing class: Keratoconus
[INFO] Extracting features for: 53
[INFO] Extracting features for: 55
[INFO] Extracting features for: 56
[INFO] Extracting features for: 57
[INFO] Extracting features for: 59
[INFO] Extracting features for: 61
[INFO] Extracting features for: 62
[INFO] Extracting features for: 65
[INFO] Extracting features for: 66
[INFO] Extracting features for: 67
[INFO] Completed extracting features for class Keratoconus

[INFO] Processing class: normal
[INFO] Extracting features for: 207
[INFO] Extracting features for: 208
[INFO] Extracting features for: 209
[INFO] Extracting features for: 210
[INFO] Extracting features for: 211
[INFO] Extracting features for: 212
[INFO] Extracting features for: 213
[INFO] Extracting features for: 214
[INFO] Extracting features for: 215
[INFO] Extracting features for: 216
[INFO] Completed extracting features for class normal


In [ ]:
import os
import re
import numpy as np
import torch
import cv2
import easyocr
from transformers import AutoTokenizer, AutoModel


class TextModule:
    
    def __init__(self, 
                 transformer_model="sentence-transformers/all-MiniLM-L6-v2",
                 device=None):

        # Device
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device

        # OCR engine
        self.reader = easyocr.Reader(['en'])

        # Transformer encoder
        self.tokenizer = AutoTokenizer.from_pretrained(transformer_model)
        self.model = AutoModel.from_pretrained(transformer_model).to(self.device)
        self.model.eval()


    # ---------------------------------------------------------
    # 1. OCR Extraction
    # ---------------------------------------------------------
    def extract_text(self, text_panel_img):
        """
        Input: RGB image crop of text panel
        Output: clean concatenated text
        """
        result = self.reader.readtext(text_panel_img)
        lines = [res[1] for res in result]
        raw_text = "\n".join(lines)
        return raw_text.strip()


    # ---------------------------------------------------------
    # 2. Clinical Numeric Feature Extraction
    # ---------------------------------------------------------
    def extract_clinical_features(self, text):
        """
        Extract important Pentacam numeric values using regex.
        If not found, returns None.
        """

        clinical = {}

        patterns = {
            "k1":  r"K1[:\s]*([0-9.]+)",
            "k2":  r"K2[:\s]*([0-9.]+)",
            "kmax": r"Kmax[:\s]*([0-9.]+)",
            "astig": r"Astig[:\s]*([0-9.\-]+)",
            "pachy": r"Pachy[:\s]*([0-9]+)",
            "thinnest": r"Thin[:\s]*([0-9]+)"
        }

        for key, pat in patterns.items():
            match = re.search(pat, text, flags=re.IGNORECASE)
            if match:
                clinical[key] = float(match.group(1))
            else:
                clinical[key] = None

        return clinical


    # ---------------------------------------------------------
    # 3. Transformer Text Embedding
    # ---------------------------------------------------------
    def text_embedding(self, text):
        """
        Convert text into dense embedding vector (CLS token).
        """

        if text.strip() == "":
            return np.zeros(384, dtype=np.float32)  # embedding size for MiniLM

        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(self.device)

        with torch.no_grad():
            outputs = self.model(**inputs)

        # CLS token embedding
        emb = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
        return emb.astype(np.float32)


    # ---------------------------------------------------------
    # 4. Complete text processing (OCR → clinical → embedding)
    # ---------------------------------------------------------
    def process_text_panel(self, text_panel_img, save_path=None, img_name=None):
        """
        Input: RGB image (text panel crop)
        Output: structured dict:
        {
           "raw_text": "...",
           "clinical_features": {...},
           "text_embedding": [...]
        }
        """

        # 1. Raw OCR text
        raw_text = self.extract_text(text_panel_img)

        # 2. Clinical numeric values
        clinical_features = self.extract_clinical_features(raw_text)

        # 3. Transformer text vector
        embedding = self.text_embedding(raw_text)

        output = {
            "raw_text": raw_text,
            "clinical_features": clinical_features,
            "text_embedding": embedding.tolist()
        }

        # 4. Optional saving
        if save_path is not None:
            os.makedirs(save_path, exist_ok=True)
            if img_name is None:
                img_name = "unknown"

            out_file = os.path.join(save_path, f"{img_name}_text.json")
            import json
            with open(out_file, "w") as f:
                json.dump(output, f, indent=4)

        return output


In [ ]:
# class FusionModule:
#     def __init__(self, xgb_model):
#         self.model = xgb_model
# 
#     def build_fusion_vector(self, image_dict, text_dict): ...
#     def run(self, Image_Output_Dict, Text_Output_Dict):
#         x = self.build_fusion_vector(Image_Output_Dict, Text_Output_Dict)
#         pred = self.model.predict(x)
#         conf = self.model.predict_proba(x)
#         return Fusion_Output_Dict


In [ ]:
# class ReportModule:
#     def __init__(self, t5_model, tokenizer):
#         self.model = t5_model
#         self.tokenizer = tokenizer
# 
#     def build_long_text(self, image_dict, text_dict, fusion_dict): ...
#     def summarize(self, text): ...
# 
#     def run(self, Image_Output_Dict, Text_Output_Dict, Fusion_Output_Dict):
#         full = self.build_long_text(...)
#         summary = self.summarize(full)
#         return summary


In [ ]:
# def run_pipeline(image_path):
#     # 1 Vision
#     Image_Output_Dict, OCR_crop = VisionModule.run(image_path)
# 
#     # 2 Text
#     Text_Output_Dict = TextModule.run(OCR_crop)
# 
#     # 3 Fusion
#     Fusion_Output_Dict = FusionModule.run(Image_Output_Dict, Text_Output_Dict)
# 
#     # 4 Report
#     final_report = ReportModule.run(Image_Output_Dict, Text_Output_Dict, Fusion_Output_Dict)
# 
#     return {
#         "vision": Image_Output_Dict,
#         "text": Text_Output_Dict,
#         "fusion": Fusion_Output_Dict,
#         "report": final_report
#     }
